In [315]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.seasonal import seasonal_decompose

In [316]:
filename = "Houses.pkl"
seed = 0

In [317]:
f = open(filename,"rb")
df = pickle.load(f)
X_feats = pickle.load(f)
Y_col = pickle.load(f)
f.close()

In [318]:
df.head()

,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT,sold_days_before_last_sale,extra_rooms,MPlus
5,5,3,2,Hot Water Rad,Y,1.0,10,5,1913.0,NaN,...,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW,3058 days,5,True
16,16,3,1,Warm Cool,Y,1.0,8,3,1908.0,2006.0,...,4201.0,004201 2005,Ward 2,152,-77.040429,38.914881,NW,2807 days,5,True
20,20,3,1,Hot Water Rad,Y,1.0,14,5,1880.0,2009.0,...,5201.0,005201 4003,Ward 2,208,-77.033366,38.911716,NW,912 days,9,False
23,23,2,1,Forced Air,Y,1.0,5,3,1880.0,2010.0,...,5201.0,005201 4003,Ward 2,208,-77.033366,38.911716,NW,2151 days,2,True
24,24,2,1,Hot Water Rad,Y,1.0,8,3,1880.0,2004.0,...,5201.0,005201 4003,Ward 2,208,-77.033366,38.911716,NW,2178 days,5,True


In [319]:
df[X_feats].head()

,sold_days_before_last_sale,BEDRM,BATHRM,HF_BATHRM,extra_rooms
5,3058 days,5,3,2,5
16,2807 days,3,3,1,5
20,912 days,5,3,1,9
23,2151 days,3,2,1,2
24,2178 days,3,2,1,5


In [320]:
df[Y_col].head()

5      True
16     True
20    False
23     True
24     True
Name: MPlus, dtype: bool

In [321]:
X = df[X_feats]
for col in X.columns:
    X[col] = (X[col] - min(X[col]))/(max(X[col]) - min(X[col]))

/home/sean/Code/incoming/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [322]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [323]:
logreg = LogisticRegression(fit_intercept = False, C = 1e12, solver='lbfgs')
model_log = logreg.fit(X_train, y_train)
model_log

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
          fit_intercept=False, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [324]:
y_hat_prob = logreg.predict_proba(X_test)

In [325]:
y_hat_prob[:20]

array([[0.56066639, 0.43933361],
       [0.8569717 , 0.1430283 ],
       [0.76137688, 0.23862312],
       [0.79272744, 0.20727256],
       [0.96309772, 0.03690228],
       [0.48650568, 0.51349432],
       [0.89936178, 0.10063822],
       [0.51550269, 0.48449731],
       [0.87482266, 0.12517734],
       [0.87169499, 0.12830501],
       [0.67558232, 0.32441768],
       [0.85174903, 0.14825097],
       [0.83137395, 0.16862605],
       [0.41895045, 0.58104955],
       [0.46110484, 0.53889516],
       [0.9390607 , 0.0609393 ],
       [0.64324516, 0.35675484],
       [0.29901705, 0.70098295],
       [0.52064812, 0.47935188],
       [0.79778164, 0.20221836]])

In [326]:
y_test.head(20)

40976      True
39124     False
53783     False
64726     False
94330     False
38662      True
26173      True
32461      True
85689     False
46759     False
84158     False
61371     False
8465       True
57149     False
53231     False
58209     False
103106    False
11332     False
97369     False
103931    False
Name: PRICE, dtype: bool

In [335]:
loglike = 0
for i, tf in enumerate(y_test):
    if tf:
        loglike += np.log(y_hat_prob[i][1])
    else:
        loglike += np.log(y_hat_prob[i][0])
print(loglike)

-3118.8616457355


In [331]:
k = len(X_feats)
print(k)

5


In [337]:
AIC = 2*k - 2*loglike
print(AIC)

6247.723291471


In [338]:
lowest_AIC = 6247.723291471